In [ ]:
from tqdm import tqdm
import math
class Optimizer:
  def __init__(self, loss_function, X_train, y_train, activation_fn, layers_dimensions ,method, batch_size, epochs,validX_train = None, validy_train = None,weight_decay = 0, eta = 0.01, beta = 0.9, beta2 = 0.999, regularization_fn = "L2",grad_reglr_fn = "L2_d" , use_wandb=False):
      self.loss_function = loss_function
      self.X_train = X_train
      self.y_train = y_train
      self.validX_train = validX_train
      self.validy_train = validy_train
      self.activation_fn = activation_fn
      self.method = method
      self.n = layers_dimensions
      self.batch_size = batch_size
      self.epochs = epochs
      self.weight_decay = weight_decay
      self.eta = eta
      self.epsilon =1e-10
      self.beta = beta
      self.beta2 = beta2
      self.regularization_fn = regularization_fn
      self.grad_reglr_fn = grad_reglr_fn
      self.batches_number = self.X_train.shape[1]//self.batch_size
      z = Weight_bias(self.n, self.activation_fn, self.y_train, self.method)
      self.init_network = z.Init_network()
      self.min_eta = 1e-4
      self.use_wandb = use_wandb
      # wandb.init(project= "DA6401_Assignment_1" , name = "RMSProp with validation data of 1/3rd of 60k rows")


class SGD(Optimizer):

  def Gradient_descent(self):


    assert(self.X_train.shape[1] == self.y_train.shape[1])
    if self.validy_train is not None:
      assert(self.validX_train.shape[1] == self.validy_train.shape[1])
    L = len(self.n) - 1
    # Initialize variables neesed to keep track of loss
    self.eta_history = []
    self.loss_history = []
    self.training_history = []
    self.valid_loss_history = []
    self.validation_history = []
    overall_loss = 0
    accuracy = 0
    val_accuracy = 0
    val_overall_loss = 0
    self.batch = 0
    accuracy = 0
    val_accuracy = 0
    val_overall_loss = 0
    train_loss = 0

    for epochs in range(self.epochs):

      for batch in tqdm(range(self.batches_number)):
        print(f"   batch no: {batch+1}     Train acc: {accuracy}     val acc: {val_accuracy}    Train Loss: {train_loss}" )

        X_batch = self.X_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.X_batch = X_batch
        # print("X_batch shape : " , self.X_batch.shape)

        y_true_batch = self.y_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.y_true_batch = y_true_batch
        fw_network = Feedforward(self.X_batch,self.activation_fn,self.method, self.init_network).Forward_prop()
        self.fw_network = fw_network
        self.y_predicted = self.fw_network[L-1].h
        self.y_pred_batch = self.y_predicted
        # self.y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # print("y_pred_batch shape : " , self.y_pred_batch.shape)
        # print("y_true_batch shape : " , self.y_true_batch.shape)
        assert(self.y_true_batch.shape[1] == self.y_pred_batch.shape[1])

        self.loss = callloss(self.loss_function, self.y_true_batch, self.y_pred_batch).give_loss()
        overall_loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.fw_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
        self.loss_history.append(overall_loss)
        bp_network = Backpropagation(self.loss_function, self.X_batch, self.y_true_batch, self.y_pred_batch, self.fw_network, self.weight_decay, self.batch, self.batch_size, self.activation_fn ).backward_propagation()
        self.bp_network = bp_network

        if len(self.loss_history) > 5:  # Ensure we have enough data points
          recent_losses = self.loss_history[-5:]
          loss_std = np.std(recent_losses)  # Compute standard deviation of recent losses

          if self.loss_history[-1] > self.loss_history[-2]:
              self.eta = max(self.eta * (0.9 if loss_std < 0.01 else 0.8), self.min_eta)

        self.eta_history.append(self.eta)

        for j in range(L):
          self.fw_network[j].weight -= self.eta * self.bp_network[j].d_w
          self.fw_network[j].bias -= self.eta * self.bp_network[j].d_b

        acc, loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
        accuracy = round(acc , 4)
        train_loss = round(loss , 4)
        self.training_history.append(CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())


        if self.validX_train is not None:
          network = Feedforward(self.validX_train,self.activation_fn,self.method, self.init_network).Forward_prop()
          self.network = network
          self.valy_predicted = self.network[L-1].h
          val_acc, v_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
          val_accuracy = round(val_acc , 4)
          val_loss = round(v_loss , 4)
          val_overall_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
          self.validation_history.append(CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())


        wandb.log({'epoch': epochs,  # Use current epoch, not total epochs
                    'validation_accuracy': val_accuracy if self.validX_train is not None else 0,
                    'accuracy': accuracy,
                    'validation_loss': val_loss if self.validX_train is not None else 0,
                    'loss': train_loss
                })

    return self.loss_history, self.training_history, self.eta_history , self.validation_history



class MGD(Optimizer):

  def momentum_GD(self):

    assert(self.X_train.shape[1] == self.y_train.shape[1])
    if self.validy_train is not None:
      assert(self.validX_train.shape[1] == self.validy_train.shape[1])
    L = len(self.n) - 1
    # Initialize variables neesed to keep track of loss
    self.eta_history = []
    self.loss_history = []
    self.training_history = []
    self.valid_loss_history = []
    self.validation_history = []
    overall_loss = 0
    self.batch = 0
    accuracy = 0
    val_accuracy = 0
    val_overall_loss = 0
    train_loss = 0

    u_w = [np.zeros_like(self.init_network[k].weight) for k in range(L)]
    u_b = [np.zeros_like(self.init_network[k].bias) for k in range(L)]

    for epochs in range(self.epochs):

      for batch in tqdm(range(self.batches_number)):
        print(f"   batch no: {batch+1}     Train acc: {accuracy}     val acc: {val_accuracy}    Train Loss: {train_loss}" )

        X_batch = self.X_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.X_batch = X_batch
        # print("X_batch shape : " , self.X_batch.shape)

        y_true_batch = self.y_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.y_true_batch = y_true_batch
        fw_network = Feedforward(self.X_batch,self.activation_fn,self.method, self.init_network).Forward_prop()
        self.fw_network = fw_network
        self.y_predicted = self.fw_network[L-1].h
        self.y_pred_batch = self.y_predicted
        # self.y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # print("y_pred_batch shape : " , self.y_pred_batch.shape)
        # print("y_true_batch shape : " , self.y_true_batch.shape)
        assert(self.y_true_batch.shape[1] == self.y_pred_batch.shape[1])

        self.loss = callloss(self.loss_function, self.y_true_batch, self.y_pred_batch).give_loss()
        overall_loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.fw_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
        self.loss_history.append(overall_loss)
        bp_network = Backpropagation(self.loss_function, self.X_batch, self.y_true_batch, self.y_pred_batch, self.fw_network, self.weight_decay, self.batch, self.batch_size, self.activation_fn ).backward_propagation()
        self.bp_network = bp_network

        if len(self.loss_history) > 5:  # Ensure we have enough data points
          recent_losses = self.loss_history[-5:]
          loss_std = np.std(recent_losses)  # Compute standard deviation of recent losses

          if self.loss_history[-1] > self.loss_history[-2]:
              self.eta = max(self.eta * (0.9 if loss_std < 0.01 else 0.8), self.min_eta)

        self.eta_history.append(self.eta)
        # self.beta = min(1 - 2 ** (-1 - math.log((self.calls / 250.0) + 1, 2)), self.beta)

        for j in range(L):
          u_w[j] = u_w[j] * self.beta + self.bp_network[j].d_w * self.eta
          u_b[j] = u_b[j] * self.beta + self.bp_network[j].d_b * self.eta

          self.fw_network[j].weight -= u_w[j]
          self.fw_network[j].bias -= u_b[j]

        acc, loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
        accuracy = round(acc , 4)
        train_loss = round(loss , 4)
        self.training_history.append(CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())


        if self.validX_train is not None:
          network = Feedforward(self.validX_train,self.activation_fn,self.method, self.init_network).Forward_prop()
          self.network = network
          self.valy_predicted = self.network[L-1].h
          val_acc, v_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
          val_accuracy = round(val_acc , 4)
          val_loss = round(v_loss , 4)
          val_overall_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
          self.validation_history.append(CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())


        # wandb.log({'epoch': epochs,  # Use current epoch, not total epochs
        #             'validation_accuracy': val_accuracy if self.validX_train is not None else 0,
        #             'accuracy': accuracy,
        #             'validation_loss': val_loss if self.validX_train is not None else 0,
        #             'loss': train_loss
        #         })

    return self.loss_history, self.training_history, self.eta_history , self.validation_history




class NAG(Optimizer):

  def Nesterov_AGD(self):

    assert(self.X_train.shape[1] == self.y_train.shape[1])
    if self.validy_train is not None:
      assert(self.validX_train.shape[1] == self.validy_train.shape[1])
    L = len(self.n) - 1
    # Initialize variables neesed to keep track of loss
    self.eta_history = []
    self.loss_history = []
    self.training_history = []
    self.valid_loss_history = []
    self.validation_history = []
    overall_loss = 0
    self.batch = 0
    accuracy = 0
    val_accuracy = 0
    val_overall_loss = 0
    train_loss = 0
    u_w = [np.zeros_like(self.init_network[k].weight) for k in range(L)]
    u_b = [np.zeros_like(self.init_network[k].bias) for k in range(L)]

    lookleap_network = self.init_network[:]

    for epochs in range(self.epochs):

      for batch in tqdm(range(self.batches_number)):
        print(f"   batch no: {batch+1}     Train acc: {accuracy}     val acc: {val_accuracy}    Train Loss: {train_loss}" )

        X_batch = self.X_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.X_batch = X_batch
        # print("X_batch shape : " , self.X_batch.shape)

        y_true_batch = self.y_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.y_true_batch = y_true_batch
        fw_network = Feedforward(self.X_batch,self.activation_fn,self.method, self.init_network).Forward_prop()
        self.fw_network = fw_network
        self.y_predicted = self.fw_network[L-1].h
        self.y_pred_batch = self.y_predicted
        # self.y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # print("y_pred_batch shape : " , self.y_pred_batch.shape)
        # print("y_true_batch shape : " , self.y_true_batch.shape)
        assert(self.y_true_batch.shape[1] == self.y_pred_batch.shape[1])

        self.loss = callloss(self.loss_function, self.y_true_batch, self.y_pred_batch).give_loss()
        overall_loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.fw_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
        self.loss_history.append(overall_loss)
        bp_network = Backpropagation(self.loss_function, self.X_batch, self.y_true_batch, self.y_pred_batch, self.fw_network, self.weight_decay, self.batch, self.batch_size, self.activation_fn ).backward_propagation()
        self.bp_network = bp_network

        if len(self.loss_history) > 5:  # Ensure we have enough data points
          recent_losses = self.loss_history[-5:]
          loss_std = np.std(recent_losses)  # Compute standard deviation of recent losses

          if self.loss_history[-1] > self.loss_history[-2]:
              self.eta = max(self.eta * (0.9 if loss_std < 0.01 else 0.8), self.min_eta)

        self.eta_history.append(self.eta)

        for j in range(L):
          u_w[j] = u_w[j] * self.beta + self.bp_network[j].d_w * self.eta
          u_b[j] = u_b[j] * self.beta + self.bp_network[j].d_b * self.eta

          self.fw_network[j].weight -= u_w[j]
          self.fw_network[j].bias -= u_b[j]

          lookleap_network[j].weight -= (self.eta * self.bp_network[j].d_w + self.beta * u_w[j])
          lookleap_network[j].bias -= (self.eta * self.bp_network[j].d_b + self.beta * u_b[j])

        acc, loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
        accuracy = round(acc , 4)
        train_loss = round(loss , 4)
        self.training_history.append(CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())


        if self.validX_train is not None:
          network = Feedforward(self.validX_train,self.activation_fn,self.method, self.init_network).Forward_prop()
          self.network = network
          self.valy_predicted = self.network[L-1].h
          val_acc, v_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
          val_accuracy = round(val_acc , 4)
          val_loss = round(v_loss , 4)
          val_overall_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
          self.validation_history.append(CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())


        # wandb.log({'epoch': epochs,  # Use current epoch, not total epochs
        #             'validation_accuracy': val_accuracy if self.validX_train is not None else 0,
        #             'accuracy': accuracy,
        #             'validation_loss': val_loss if self.validX_train is not None else 0,
        #             'loss': train_loss
        #         })
    return self.loss_history, self.training_history, self.eta_history , self.validation_history



class RMSProp(Optimizer):

  def rms_GD(self):

    assert(self.X_train.shape[1] == self.y_train.shape[1])
    if self.validy_train is not None:
      assert(self.validX_train.shape[1] == self.validy_train.shape[1])
    L = len(self.n) - 1
    # Initialize variables neesed to keep track of loss
    self.eta_history = []
    self.loss_history = []
    self.training_history = []
    self.valid_loss_history = []
    self.validation_history = []
    overall_loss = 0
    self.batch = 0
    accuracy = 0
    val_accuracy = 0
    val_overall_loss = 0
    train_loss = 0
    u_w = [np.zeros_like(self.init_network[k].weight) for k in range(L)]
    u_b = [np.zeros_like(self.init_network[k].bias) for k in range(L)]

    for epochs in range(self.epochs):

      for batch in tqdm(range(self.batches_number)):
        print(f"   batch no: {batch+1}     Train acc: {accuracy}     val acc: {val_accuracy}    Train Loss: {train_loss}" )

        X_batch = self.X_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.X_batch = X_batch
        # print("X_batch shape : " , self.X_batch.shape)

        y_true_batch = self.y_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.y_true_batch = y_true_batch
        fw_network = Feedforward(self.X_batch,self.activation_fn,self.method, self.init_network).Forward_prop()
        self.fw_network = fw_network
        self.y_predicted = self.fw_network[L-1].h
        self.y_pred_batch = self.y_predicted
        # self.y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # print("y_pred_batch shape : " , self.y_pred_batch.shape)
        # print("y_true_batch shape : " , self.y_true_batch.shape)
        assert(self.y_true_batch.shape[1] == self.y_pred_batch.shape[1])

        self.loss = callloss(self.loss_function, self.y_true_batch, self.y_pred_batch).give_loss()
        overall_loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.fw_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
        self.loss_history.append(overall_loss)
        bp_network = Backpropagation(self.loss_function, self.X_batch, self.y_true_batch, self.y_pred_batch, self.fw_network, self.weight_decay, self.batch, self.batch_size, self.activation_fn ).backward_propagation()
        self.bp_network = bp_network

        if len(self.loss_history) > 5:  # Ensure we have enough data points
          recent_losses = self.loss_history[-5:]
          loss_std = np.std(recent_losses)  # Compute standard deviation of recent losses

          if self.loss_history[-1] > self.loss_history[-2]:
              self.eta = max(self.eta * (0.9 if loss_std < 0.01 else 0.8), self.min_eta)

        self.eta_history.append(self.eta)

        for j in range(L):
          u_w[j] = u_w[j] * self.beta +  (1  - self.beta) * (self.bp_network[j].d_w)**2
          u_b[j] = u_b[j] * self.beta +  (1  - self.beta) * (self.bp_network[j].d_b)**2

          self.fw_network[j].weight -= (self.eta / np.sqrt(u_w[j] + self.epsilon)) * self.bp_network[j].d_w
          self.fw_network[j].bias -= (self.eta / np.sqrt(u_b[j] + self.epsilon)) * self.bp_network[j].d_b

        acc, loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
        accuracy = round(acc , 4)
        train_loss = round(loss , 4)
        self.training_history.append(CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())

        if self.validX_train is not None:
          network = Feedforward(self.validX_train,self.activation_fn,self.method, self.init_network).Forward_prop()
          self.network = network
          self.valy_predicted = self.network[L-1].h
          val_acc, v_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
          val_accuracy = round(val_acc , 4)
          val_loss = round(v_loss , 4)
          val_overall_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
          self.validation_history.append(CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())


        # wandb.log({'epoch': epochs,  # Use current epoch, not total epochs
        #             'validation_accuracy': val_accuracy if self.validX_train is not None else 0,
        #             'accuracy': accuracy,
        #             'validation_loss': val_loss if self.validX_train is not None else 0,
        #             'loss': train_loss
        #         })

    return self.loss_history, self.training_history, self.eta_history , self.validation_history


class ADAM(Optimizer):

  def adam_GD(self):

    assert(self.X_train.shape[1] == self.y_train.shape[1])
    if self.validy_train is not None:
      assert(self.validX_train.shape[1] == self.validy_train.shape[1])
    L = len(self.n) - 1
    # Initialize variables neesed to keep track of loss
    self.eta_history = []
    self.loss_history = []
    self.training_history = []
    self.valid_loss_history = []
    self.validation_history = []
    overall_loss = 0
    self.batch = 0
    accuracy = 0
    val_accuracy = 0
    val_overall_loss = 0
    train_loss = 0

    i = 0
    u_w = [np.zeros_like(self.init_network[k].weight) for k in range(L)]
    u_b = [np.zeros_like(self.init_network[k].bias) for k in range(L)]
    v_w = [np.zeros_like(self.init_network[k].weight) for k in range(L)]
    v_b = [np.zeros_like(self.init_network[k].bias) for k in range(L)]

    for epochs in range(self.epochs):

      for batch in tqdm(range(self.batches_number)):
        print(f"   batch no: {batch+1}     Train acc: {accuracy}     val acc: {val_accuracy}    Train Loss: {train_loss}" )

        X_batch = self.X_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.X_batch = X_batch
        # print("X_batch shape : " , self.X_batch.shape)

        y_true_batch = self.y_train[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        self.y_true_batch = y_true_batch
        fw_network = Feedforward(self.X_batch,self.activation_fn,self.method, self.init_network).Forward_prop()
        self.fw_network = fw_network
        self.y_predicted = self.fw_network[L-1].h
        self.y_pred_batch = self.y_predicted
        # self.y_pred_batch = self.y_predicted[:, batch*self.batch_size:(batch+1)*self.batch_size]
        # print("y_pred_batch shape : " , self.y_pred_batch.shape)
        # print("y_true_batch shape : " , self.y_true_batch.shape)
        assert(self.y_true_batch.shape[1] == self.y_pred_batch.shape[1])

        self.loss = callloss(self.loss_function, self.y_true_batch, self.y_pred_batch).give_loss()
        overall_loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.fw_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
        self.loss_history.append(overall_loss)
        bp_network = Backpropagation(self.loss_function, self.X_batch, self.y_true_batch, self.y_pred_batch, self.fw_network, self.weight_decay, self.batch, self.batch_size, self.activation_fn ).backward_propagation()
        self.bp_network = bp_network

        if len(self.loss_history) > 5:  # Ensure we have enough data points
          recent_losses = self.loss_history[-5:]
          loss_std = np.std(recent_losses)  # Compute standard deviation of recent losses

          if self.loss_history[-1] > self.loss_history[-2]:
              self.eta = max(self.eta * (0.9 if loss_std < 0.01 else 0.8), self.min_eta)

        self.eta_history.append(self.eta)

        for j in range(L):
          u_w[j] = u_w[j] * self.beta +  (1  - self.beta) * (self.bp_network[j].d_w)
          u_b[j] = u_b[j] * self.beta +  (1  - self.beta) * (self.bp_network[j].d_b)
          v_w[j] = v_w[j] * self.beta2 +  (1  - self.beta2) * (self.bp_network[j].d_w)**2
          v_b[j] = v_b[j] * self.beta2 +  (1  - self.beta2) * (self.bp_network[j].d_b)**2

          u_w_pred = u_w[j] / (1 - np.power(self.beta , i+1))
          u_b_pred = u_b[j] / (1 - np.power(self.beta , i+1))
          v_w_pred = v_w[j] / (1 - np.power(self.beta2 , i+1))
          v_b_pred = v_b[j] / (1 - np.power(self.beta2 , i+1))

          self.fw_network[j].weight -= (self.eta / (np.sqrt(v_w_pred) + self.epsilon)) * u_w_pred
          self.fw_network[j].bias -= (self.eta / (np.sqrt(v_b_pred) + self.epsilon)) * u_b_pred

        self.init_network = self.fw_network

        acc, loss = CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
        accuracy = round(acc , 4)
        train_loss = round(loss , 4)
        self.training_history.append(CalculateAllLoss(self.X_batch, self.y_pred_batch, self.init_network, self.y_true_batch, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())
        i+=1

        if self.validX_train is not None:
          network = Feedforward(self.validX_train,self.activation_fn,self.method, self.init_network).Forward_prop()
          self.network = network
          self.valy_predicted = self.network[L-1].h
          val_acc, v_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss()
          val_accuracy = round(val_acc , 4)
          val_loss = round(v_loss , 4)
          val_overall_loss = CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).overall_loss()
          self.validation_history.append(CalculateAllLoss(self.validX_train, self.valy_predicted, self.init_network, self.validy_train, self.loss, self.weight_decay, self.regularization_fn).calc_accuracy_loss())


        # wandb.log({'epoch': epochs,  # Use current epoch, not total epochs
        #             'validation_accuracy': val_accuracy if self.validX_train is not None else 0,
        #             'accuracy': accuracy,
        #             'validation_loss': val_loss if self.validX_train is not None else 0,
        #             'loss': train_loss
        #         })

    return self.loss_history, self.training_history, self.eta_history , self.validation_history


class GiveOptimizers(Optimizer):
    def __init__(self, optimization_fn, loss_function, X_train, y_train, activation_fn, layers_dimensions ,method, batch_size, epochs,  validX_train , validy_train ,weight_decay = 0, eta = 0.01, beta = 0.9, beta2 = 0.999, regularization_fn = "L2",grad_reglr_fn = "L2_d", use_wandb=False):
      self.optimization_function = optimization_fn.lower()
      super().__init__( loss_function, X_train, y_train, activation_fn, layers_dimensions ,method, batch_size, epochs, validX_train, validy_train ,weight_decay = 0, eta = 0.01, beta = 0.9, beta2 = 0.999, regularization_fn = "L2",grad_reglr_fn = "L2_d" , use_wandb=False)

    def apply_optimization(self):
      if self.optimization_function == 'sgd':
          return SGD(self.loss_function, self.X_train, self.y_train, self.activation_fn, self.n, self.method, self.batch_size, self.epochs,  self.validX_train, self.validy_train, self.weight_decay,self.eta, self.beta, self.beta2, self.regularization_fn, self.grad_reglr_fn, self.use_wandb).Gradient_descent()
      elif self.optimization_function == 'momentum':
          return MGD(self.loss_function, self.X_train, self.y_train, self.activation_fn, self.n, self.method, self.batch_size, self.epochs, self.validX_train, self.validy_train, self.weight_decay,self.eta, self.beta, self.beta2, self.regularization_fn, self.grad_reglr_fn, self.use_wandb).momentum_GD()
      elif self.optimization_function == 'nag':
          return NAG(self.loss_function, self.X_train, self.y_train, self.activation_fn, self.n, self.method, self.batch_size, self.epochs, self.validX_train, self.validy_train, self.weight_decay,self.eta, self.beta, self.beta2, self.regularization_fn, self.grad_reglr_fn, self.use_wandb).Nesterov_AGD()
      elif self.optimization_function == 'rmsp':
          return RMSProp(self.loss_function, self.X_train, self.y_train, self.activation_fn, self.n, self.method, self.batch_size, self.epochs, self.validX_train, self.validy_train, self.weight_decay,self.eta, self.beta, self.beta2, self.regularization_fn, self.grad_reglr_fn, self.use_wandb).rms_GD()
      elif self.optimization_function == 'adam':
          return ADAM(self.loss_function, self.X_train, self.y_train, self.activation_fn, self.n, self.method, self.batch_size, self.epochs, self.validX_train, self.validy_train, self.weight_decay,self.eta, self.beta, self.beta2, self.regularization_fn, self.grad_reglr_fn, self.use_wandb).adam_GD()